In [99]:
%matplotlib inline

In [100]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from nose.tools import *

np.random.seed(24680)

Write your imports in the cell below.

In [101]:
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC


from sklearn.metrics import confusion_matrix,accuracy_score,f1_score


# raise NotImplementedError()

# Ensemble Models and Support Vector Machines Lab
## Training and comparing different algorithms

Once again, we'll work with the bank dataset. This time, the data preprocessing steps have been done for you.

The goal is to try and improve our predictions (if they can be improved at all) using different types of algorithms.

### 1. Read the data (1 point)
This time you only need to read the data. The indicator variables have been separated out for you.

Read the dataset and save it in the variable `bank_data`. The target column is `y`. Use the variables `bank_attributes` and `bank_labels` to save the attributes (explanatory variables, features, predictors), and labels (`y`).

In [102]:
bank_data = pd.read_csv('data/bank.csv')

bank_attributes = bank_data.drop(columns =['y'] )
bank_labels = bank_data.y

In [103]:
assert_is_not_none(bank_data)
assert_is_not_none(bank_attributes)
assert_is_not_none(bank_labels)

### 2. Normalize the data (1 point)
Because both forests and SVMs are sensitive to non-scaled data, we need to normalize our dataset first.

Rescale all columns in `bank_attributes` so they have mean = 0 and variance = 1. You can either look at the `sklearn` docs or do this yourself. When you're ready, overwrite the `bank_attributes` column. Make sure that you don't lose the column names in the process.

In [104]:
# YOUR CODE HERE
scaler = StandardScaler()
bank_attributes = scaler.fit_transform(bank_attributes)

In [105]:
assert_is_not_none(bank_attributes)

### 3. Split the data (1 point)
Use the standard 70% / 30% split. Since this is a classification problem, be sure to stratify the split according to the `bank_labels`.

In [106]:
bank_attributes_train, bank_labels_train = None, None
bank_attributes_test, bank_labels_test = None, None



bank_attributes_train,bank_attributes_test,bank_labels_train, bank_labels_test =train_test_split(
    bank_attributes,bank_labels,
    test_size = 0.30,
    random_state = 33,
    stratify = bank_labels
)


In [107]:
bank_attributes_train.shape , bank_attributes_test.shape , bank_labels_train.shape, bank_labels_test.shape

((3164, 51), (1357, 51), (3164,), (1357,))

In [108]:
assert_is_not_none(bank_attributes_train)
assert_is_not_none(bank_labels_train)

assert_is_not_none(bank_attributes_test)
assert_is_not_none(bank_labels_test)

### 4. Prepare the cross-validation folds (1 point)
Use a stratified k-fold cross-validation split, with $k = 5$. Fit it to the train data. Save the trained cross-validator to the variable `k_fold`.

The data should already be shuffled. There's no need to shuffle it again.

In [109]:
k_fold = StratifiedKFold(n_splits = 5)

In [110]:
assert_is_not_none(k_fold)

### 5. Decision Tree (2 points)
Use cross-validation to train and optimize the hyperparameters for a decision tree classifier.

Use grid search with the following grid:
* `max_depth`: 1, 5, 7, 15, 20
* `min_samples_leaf`: 2, 5, 10, 12
* `max_leaf_nodes`: 5, 10, 20

Use the most appropriate scoring metric (remember that accuracy doesn't work in this case because the data is highly imbalanced; we need something which combines precision and recall). Use the cross-validation splits you just created.

Save the grid results in `grid_search`. Save the best classifier in `tree_classifier`.

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

In [111]:
tree_classifier = DecisionTreeClassifier()
tree_classifier.fit(bank_attributes_train,bank_labels_train)
prediction = tree_classifier.predict(bank_attributes_test)

In [112]:
tree_confusion_matrix = confusion_matrix(bank_labels_test, prediction)
tree_accuracy = accuracy_score(bank_labels_test, prediction)
tree_f1_score = f1_score(bank_labels_test, prediction)

print(f'The confusion matrix is:')
print(tree_confusion_matrix)
print(f'The accuracy score is {tree_accuracy:.2f}')
print(f'The f1 score is {tree_f1_score:.2f}')

The confusion matrix is:
[[1119   82]
 [  84   72]]
The accuracy score is 0.88
The f1 score is 0.46


In [113]:
parameters_grid = {
'max_depth':[1,5,7,15,20],
'min_samples_leaf':[2,5,10,12],
'max_leaf_nodes':[5,10,20]
}
tree_grid_search = GridSearchCV(
    estimator = tree_classifier,
    param_grid= parameters_grid,
    scoring = 'f1',
    cv = k_fold
) 

tree_grid_search.fit(bank_attributes,bank_labels)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
             estimator=DecisionTreeClassifier(),
             param_grid={'max_depth': [1, 5, 7, 15, 20],
                         'max_leaf_nodes': [5, 10, 20],
                         'min_samples_leaf': [2, 5, 10, 12]},
             scoring='f1')

In [114]:
# tree_grid_search.best_score_
print("Decision tree; best score:", tree_grid_search.best_score_)

Decision tree; best score: 0.47445101975915965


In [115]:
assert_is_not_none(grid_search)
assert_is_not_none(tree_classifier)

### 6. Random Forest (1 point)
Use cross-validation to train and optimize the hyperparameters for a random forest classifier. Use the same technique as before.

Use the following grid:
* `n_estimators`: 100, 200, 300 
* `max_depth`: 20, 50, 100

Note that this grid is on the small side but this is mainly due to performance reasons. Also note that the training will take some time.

Save the grid results in `grid_search`. Save the best classifier in `forest_classifier`.

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

Due to the relatively slow training, we've chosen low values for the parameters. The performance of the random forest will be worse than the decision tree. This is not necessarily the case in general, it's due to the parameters we've chosen to try.

In [116]:
forest_classifier = RandomForestClassifier()
# forest_classifier.fit(bank_attributes_train,bank_labels_train)

parameters_grid ={
    'n_estimators':[100,200,300],
    'max_depth':[20,50,100]
}


rf_grid_search = GridSearchCV(
    estimator = forest_classifier,
    param_grid= parameters_grid,
    scoring = 'f1',
    cv = k_fold
) 

rf_grid_search.fit(bank_attributes_train,bank_labels_train)

print("Random forest; best score:", rf_grid_search.best_score_)

Random forest; best score: 0.3478196594325626


In [117]:
assert_is_not_none(grid_search)
assert_is_not_none(forest_classifier)

### 7. Linear SVM (1 point)
Use cross-validation to train and optimize the hyperparameters for a linear support vector machine. Use the same technique as before.

Use the following grid:
* `C`: 0.1, 0.5, 0.8, 1, 1.5, 2, 6, 10, 15, 20

Note that we're choosing relatively small values for `C`. This is allowed because our data is normalized.

Save the grid results in `grid_search`. Save the best classifier in `linear_svm_classifier`. There are many ways to create a linear SVM classifier. Look up the `sklearn` docs to choose the fastest one (in terms of performance).

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

In [118]:
linear_svm_classifier = SVC(kernel = 'linear')
linear_svm_classifier.fit(bank_attributes_train,bank_labels_train)

parameters_grid = {'C':[0.1,0.25,0.8,1,1.5,2,6,10,15,20]}
lsvm_grid_search = GridSearchCV(
    estimator = linear_svm_classifier,
    param_grid = parameters_grid,
    scoring = 'f1',
    cv = k_fold
)

lsvm_grid_search.fit(bank_attributes_train,bank_labels_train)


# YOUR CODE HERE
# raise NotImplementedError()

print("Linear SVM; best score:", lsvm_grid_search.best_score_)

Linear SVM; best score: 0.2602418531748925


In [119]:
assert_is_not_none(grid_search)
assert_is_not_none(linear_svm_classifier)

### 8. Gaussian SVM (1 point)
Use cross-validation to train and optimize the hyperparameters for an SVM with a Gaussian kernel. Use the same technique as before.

Use the following grid:
* `C`: 10, 15, 20, 50, 200
* `gamma`: 0.001, 0.01, 0.1, 0.2

Note that this time we give larger values of `C` because the governing parameter here is `gamma`.

Save the grid results in `grid_search`. Save the best classifier in `gaussian_svm_classifier`.

Optionally, you can print and / or visualize the cross-validation results and the best chosen parameters.

In [120]:
gaussian_svm_classifier = SVC(kernel = 'rbf')
gaussian_svm_classifier.fit(bank_attributes_train,bank_labels_train)

parameters_grid ={
    'C':[10,15,20,50,200],
    'gamma':[0.001,0.01,0.1,0.2]
}

gsvm_grid_search = GridSearchCV(
    estimator = gaussian_svm_classifier,
    param_grid = parameters_grid,
    scoring = 'f1',
    cv = k_fold
)

gsvm_grid_search.fit(bank_attributes_train,bank_labels_train)

print("Gaussian SVM; best score:", gsvm_grid_search.best_score_)

Gaussian SVM; best score: 0.42620892834555624


In [121]:
assert_is_not_none(grid_search)
assert_is_not_none(gaussian_svm_classifier)

### 9. Compare performance on the testing data (1 point)
Now that you've trained all your models, you've got to select the best one. This should be done on the testing data.

Use the appropriate scoring metric to get the testing scores for all your models. Don't forget to pass the **testing**, not the training data. Save all scores.

Choose the best classifier, based on these scores (the one with the highest test score). Of course, this is not enough. We need to look at ROC curves, track performance through other measures, debug the sources of variance in testing results, try more hyperparameters, etc. However, this is enough for an introductory lab :).

Optionally, you can think of combining them into a boosted model but this is out of the scope of this lab.

In [122]:
tree_classifier_score = tree_grid_search.score(bank_attributes_test,bank_labels_test)
forest_classifier_score =  rf_grid_search.score(bank_attributes_test,bank_labels_test)
linear_svm_classifier_score =lsvm_grid_search.score(bank_attributes_test,bank_labels_test)
gaussian_svm_classifier_score = gsvm_grid_search.score(bank_attributes_test,bank_labels_test)


# , forest_classifier_score, linear_svm_classifier_score, gaussian_svm_classifier_score = , None, None, None
# # YOUR CODE HERE
# # # raise NotImplementedError()

print("Testing scores:")
print(f"Decision tree: {tree_classifier_score:.2f}")
print(f"Random forest: {forest_classifier_score:.2f}")
print(f"Linear SVM: {linear_svm_classifier_score:.2f}")
print(f"Gaussian SVM: {gaussian_svm_classifier_score:.2f}")

Testing scores:
Decision tree: 0.62
Random forest: 0.31
Linear SVM: 0.27
Gaussian SVM: 0.42


In [123]:
best_classifier = "tree" 
# Replace empty string with "tree", "forest", "linear SVM" or "gaussian SVM"


In [124]:
assert_not_equal(best_classifier, "")